# BASE

In [1]:
# %run summarizationBackbone.ipynb
%run summarizationBackboneDocLvl.ipynb

C:\Users\Specialist\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Big File...
Finished Loading Big File.


In [3]:
from my_package.text_processing import *
from tqdm import tqdm

In [4]:
def predict(text, percentage):
    try:
        sentences, filtered_sentences = preprocessing_text_with_spacy(text,True,False)
        df = buildDF(filtered_sentences, sentences)
        summarizedRow = {"Original Article": ds[articleCol][i]}
        for key in df.keys():
            element = summarizeWith(sentences, df, key, percentage)
            summarizedRow[key] = element
        return summarizedRow
    except Exception as e:
        print("Error",i, e)

# Summarization Settings

In [5]:
path = r'DUC\duc\Duc_dataset_first_ref_summary.csv'
ds = pd.read_csv(path)

In [7]:
# Dataset Column Names
articleCol = "Original Article"
summaryCol = "Original Summary"
# Documents to be summarized
start = 0
end = len(ds)
# Summarization Percentage [0-1]% if > 1 acts as number of sentences
percentage = 5
# Pre processing settings
lemmatization = True
remove_stopwords = True
# Normalization Type
isNormOnDataset = True 
# output file name [MAKE SURE TO CHANGE TO NOT OVERWRITE]
outFileName = "DUC_1st_5Sent_lemT_swT.csv"
ppFileName = "PP_DUC_1st_5Sent_lemT_swT.csv"
ppSpecial = "#!@!#"
# Weights Settings
useWeights = True
weights = {"tm":0.92,"luhn":1,"lsa":0.44,"tr":0.84,"lex":0.88,"lda":0.8}
# doc scoring
docScoringFolder = "BBC_Sentence_Score"

In [8]:
Algorithms = ["Tm","Lex","Luhn","Lsa","Tr","LDA"]
lstCols = get_combinations(Algorithms)
cols = ["Original Article","Original Summary"] + lstCols
df3 = pd.DataFrame(columns = cols)
df3.to_csv(outFileName, index=False)

## Pre Processing

In [9]:
# sentences, processed_sentences = process_one_column_df(ds[articleCol],lemmatization,remove_stopwords)

 47%|████▋     | 23/49 [01:08<01:18,  3.01s/it]

In [ ]:
# collected = []
# for i in range(len(sentences)):
#     collected.append([ds[articleCol][i],ds[summaryCol][i],ppSpecial.join(sentences[i]),ppSpecial.join(processed_sentences[i])])
# df = pd.DataFrame(collected, columns =[articleCol,summaryCol,"Sentences","Filtered Sent"]) 
# df.to_csv(ppFileName, index=False)
# data = pd.read_csv(ppFileName)

# one by one docs

In [ ]:
# def onebyoneSummarization(ds):
#     summarizedDataset = []
#     for i in tqdm(range(start,end,1)):
#         try:
#             # sentences, filtered_sentences = preprocessing_text_with_spacy(ds[articleCol][i],lemmatization,remove_stopwords)
#             # Read Pre Processed Data
#             data = pd.read_csv(ppFileName)
#             sentences = data["Sentences"][i].split(ppSpecial)
#             filtered_sentences = data["Filtered Sent"][i].split(ppSpecial)
#             # build combination dataframe
#             df = buildDF(filtered_sentences, sentences, useWeights, weights)

#             summarizedRow = {"Original Article": ds[articleCol][i],"Original Summary": ds[summaryCol][i]}
#             for key in df.keys():
#                 element = summarizeWith(sentences, df, key, percentage)
#                 summarizedRow[key] = element
#             pd.DataFrame.from_dict(summarizedRow, orient='index').T.to_csv(outFileName,mode='a',header=False,index=False)
#             # break
#         except Exception as e:
#             print("Error",i, e)

In [ ]:
# onebyoneSummarization(ds)

100%|██████████| 49/49 [59:45<00:00, 73.17s/it]   


# Doc scoring

In [ ]:
lstCols = Algorithms
data = pd.read_csv(ppFileName)

In [ ]:
def documentScoring(sentences,filtered_sentences):
    maxes = []
    mines = []
    for i in tqdm(range(start,end,1)):
        try:
            # sentences = data["Sentences"][i].split(ppSpecial)
            # filtered_sentences = data["Filtered Sent"][i].split(ppSpecial)
            df = buildDF(filtered_sentences, sentences, useWeights, weights)
            df.to_csv(docScoringFolder+"/Document " + str(i) + ".csv", index=False)
            maxes.append([df[Algorithms[0]].max(),df[Algorithms[1]].max(),df[Algorithms[2]].max(),df[Algorithms[3]].max(),df[Algorithms[4]].max(),df[Algorithms[5]].max()])
            mines.append([df[Algorithms[0]].min(),df[Algorithms[1]].min(),df[Algorithms[2]].min(),df[Algorithms[3]].min(),df[Algorithms[4]].min(),df[Algorithms[5]].min()])
        except Exception as e:
            print("Error",i, e)
    df = pd.DataFrame(maxes, columns = Algorithms) 
    df.to_csv(docScoringFolder+"/max.csv", index=False)
    df = pd.DataFrame(mines, columns = Algorithms) 
    df.to_csv(docScoringFolder+"/min.csv", index=False)

100%|██████████| 49/49 [52:37<00:00, 64.44s/it] 


# Normalization

In [ ]:
maxdf = pd.read_csv(docScoringFolder+"/max.csv")
mindf = pd.read_csv(docScoringFolder+"/min.csv")

In [ ]:
absmax = [maxdf[Algorithms[0]].max(),maxdf[Algorithms[1]].max(),maxdf[Algorithms[2]].max(),maxdf[Algorithms[3]].max(),maxdf[Algorithms[4]].max(),maxdf[Algorithms[5]].max()]
absmin = [mindf[Algorithms[0]].min(),mindf[Algorithms[1]].min(),mindf[Algorithms[2]].min(),mindf[Algorithms[3]].min(),mindf[Algorithms[4]].min(),mindf[Algorithms[5]].min()]

In [ ]:
def normalize(values, min_value, max_value):
    normalized_values = [(x - min_value) / (max_value - min_value) for x in values]
    return normalized_values

Main Function for dataset level norm (Merged on next cell)

In [ ]:
    # # dataset = []
    # for i in tqdm(range(start,end,1)):
    #     # try:
    #         # Normalization
    #         df = pd.read_csv("DUC Sentence Score by Document/Document "+ str(i) +".csv")
    #         norm2dlist = pd.DataFrame()
    #         norm2dlist["Sent"] = df["Sent"]
    #         for key in range(len(Algorithms)):
    #             norm2dlist[Algorithms[key]] = normalize(df[Algorithms[key]],absmin[key],absmax[key])
    #         # Combinations  
    #         combdf = allCombsDf(norm2dlist)
    #         # Summarization
    #         summarizedRow = {"Original Article": ds[articleCol][i],"Original Summary": ds[summaryCol][i]}
    #         for key in combdf.keys():
    #             element = summarizeWith(df["Sent"], combdf, key, percentage)
    #             summarizedRow[key] = element
    #         pd.DataFrame.from_dict(summarizedRow, orient='index').T.to_csv(outFileName,mode='a',header=False,index=False)
    #     # except Exception as e:
    #     #     print("Error",i, e)
    #     #     break
    #     # dataset.append(norm2dlist)

# Main Function

In [ ]:
    # dataset = []
    for i in tqdm(range(start,end,1)):
        # try:
            # Normalization
            df = pd.read_csv(docScoringFolder+"/Document "+ str(i) +".csv")
            normdf = pd.DataFrame()
            normdf["Sent"] = df["Sent"]
            for key in range(len(Algorithms)):
                if(isNormOnDataset):
                    normMax = absmax[key]
                    normMin = absmin[key]
                else:
                    normMax = df[Algorithms[key]].max()
                    normMin = df[Algorithms[key]].min()
                normdf[Algorithms[key]] = normalize(df[Algorithms[key]],normMin,normMax)
            # Combinations  
            combdf = allCombsDf(normdf)
            # Summarization
            summarizedRow = {"Original Article": ds[articleCol][i],"Original Summary": ds[summaryCol][i]}
            for key in combdf.keys():
                element = summarizeWith(df["Sent"], combdf, key, percentage)
                summarizedRow[key] = element
            pd.DataFrame.from_dict(summarizedRow, orient='index').T.to_csv(outFileName,mode='a',header=False,index=False)
        # except Exception as e:
        #     print("Error",i, e)
        #     break
        # dataset.append(norm2dlist)

100%|██████████| 49/49 [00:13<00:00,  3.67it/s]
